<a href="https://colab.research.google.com/github/jouherdauf/Web-Scrapping-Articles-Using-Website-Url-s/blob/main/Data_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd
import numpy as np
from google.colab import files
data = files.upload()

In [ ]:
#importing the input file
df = pd.read_csv('Input.csv')

In [ ]:
#dropping the row where URL iss null
df.dropna(subset=['URL'],inplace = True)

In [ ]:
# creating column where we upadate URL content in string format
df['response'] = np.nan

In [ ]:
#saving text file of URL content & updating response column
for i  in df.index:
  try:
    url = df.loc[i,"URL"]
    string = requests.get(url)
    html_content = string.content
# Parse the HTML using Beautiful Soup
    soup = BeautifulSoup(html_content, "html.parser")
    paragraphs = soup.find_all("h1")
    division = soup.find_all("div", {"class": "tagdiv-type"})
    paragraphs.extend(division)
    texts = [paragraph.get_text() for paragraph in paragraphs]
    abc = ''
    for paragrap in texts:
        abc += f'''{paragrap}'''
    df.at[i,'response'] = abc
    with open(f'{df.loc[i,"URL_ID"]}.txt', 'w',encoding="utf-8") as file:
        file.write(abc)
  except:
    df.at[i,'response'] = np.nan

In [ ]:
#copying the dataframe
df1 = df.copy()

In [ ]:
#dropping the \n from the string
for i in df.index:
    df1.loc[i,'response'] = df1.loc[i,'response'].replace('\n',' ')

#### Creating List of all stopwords and merging it

In [ ]:
StopWordsNames = []
file = open("StopWords_Names.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsNames.append(stripped_line)

In [ ]:
StopWordsGeographic = []
file = open("StopWords_Geographic.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsGeographic.append(stripped_line)

In [ ]:
StopWordsGenericLong = []
file = open("StopWords_GenericLong.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsGenericLong.append(stripped_line)

In [ ]:
StopWordsGeneric = []
file = open("StopWords_Generic.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsGeneric.append(stripped_line)

In [ ]:
StopWordsAuditor = []
file = open("StopWords_Auditor.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsAuditor.append(stripped_line)

In [ ]:
StopWordsDatesandNumbers = []
file = open("StopWords_DatesandNumbers.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsDatesandNumbers.append(stripped_line)

In [ ]:
StopWordsCurrencies = []
file = open("StopWords_Currencies.txt", "r")
for line in file:
  stripped_line = line.strip()
  StopWordsCurrencies.append(stripped_line)

In [ ]:
#merging all the stopwords
stopwordslists = [StopWordsCurrencies,StopWordsDatesandNumbers,StopWordsAuditor,StopWordsGeneric,StopWordsGenericLong,StopWordsGeographic,StopWordsNames]
stopwords = []
for i in stopwordslists:
    for j in i:
        stopwords.append(j)

In [ ]:
#removing stop word from the string
for i in df.index:
    for j in stopwords:
        df1.loc[i,'response'] = df1.loc[i,'response'].replace(' '+j+' ',' ')

In [ ]:
# removing pre tag from the string
words =''
for i in df1.index:
    x = 0
    for j in df1.loc[i,'response']:
        words += j
        x += 1
        if words[-11:] == 'Blackcoffer':
            df1.loc[i,'response'] = df1.loc[i,'response'][0:x-11]

In [ ]:
#Creating list of all words
df1['words'] = ''
import re
def split_sentence(sentence):
    return re.findall(r'\b\w+\b', sentence)
#Driver code
for i in df1.index:
    sentence = df1.loc[i,'response'][:]
    df1.at[i,'words'] = split_sentence(sentence)

In [ ]:
# counting number of words
df1['WORD COUNT'] = np.nan

for i in df1.index:
    count = 0
    for j in df1.loc[i,'words']:
        if  j[0] not in  ['0','1','2','3','4','5','6','7','8','9']:
            count += 1
            df1.loc[i,'WORD COUNT'] = count


In [ ]:
# average word length
df1['AVG WORD LENGTH'] = np.nan
for i in df1.index:
    count = 0
    for j in df1.loc[i,'words']:
        if  j[0] not in  ['0','1','2','3','4','5','6','7','8','9']:
            count += len(j)
            df1.loc[i,'AVG WORD LENGTH'] = count/df1.loc[i,'WORD COUNT']


In [ ]:
#function to count number of syllable
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [ ]:
#checking the number of complex words
words =''
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
df1['Complex_words'] = np.nan
for i in df1.index:
    x = 0
    for j in df1.loc[i,'response']:
        if j in alphabets:
            words += j
        elif words == '':
            words = ''
        else:
            if words[-2:] not in ['es','ed']:
                syllable = syllable_count(words)
                words = ''
                if syllable > 2:
                    x += 1
                    df1.at[i,'Complex_words'] = x

In [ ]:
#checking the number of average number of syllable per word
words =''
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
df1['SYLLABLE PER WORD'] = np.nan
for i in df1.index:
    x = 0
    for j in df1.loc[i,'response']:
        if j in alphabets:
            words += j
        elif words == '':
            words = ''
        else:
            if words[-2:] not in ['es','ed']:
                syllable = syllable_count(words)
                x += syllable
                df1.at[i,'SYLLABLE PER WORD'] =  x/(df1.loc[i,'WORD COUNT'])
                words = ''

In [ ]:
# NUmber of complex words
df1['Complex_words'] = np.nan
for i in df1.index:
    x = 0
    for j in df1.loc[i,'response']:
        if j in alphabets:
            words += j
        elif words == '':
            words = ''
        else:
            if words[-2:] not in ['es','ed']:
                syllable = syllable_count(words)
                words = ''
                if syllable > 2:
                    x += 1
                    df1.at[i,'Complex_words'] = x

In [ ]:
#Checking percentage of complex words
df1['Percentage OF COMPLEX WORDS'] = (df1['Complex_words']/df1['WORD COUNT'])*100

In [ ]:
# creating list of positive words
positivewords = []
file = open("positive-words.txt", "r")
for line in file:
  stripped_line = line.strip()
  positivewords.append(stripped_line)

In [ ]:
# creating list of negative words
negativewords = []
file = open("negative-words.txt", "r")
for line in file:
  stripped_line = line.strip()
  negativewords.append(stripped_line)

In [ ]:
# creating  a column of negative words
for i in df1.index:
    x =0
    for j in df1.loc[i,'words']:
        if j in positivewords:
            x += 1
            df1.loc[i,'POSITIVE WORDS'] = x

In [ ]:
#creating a column of positive words
for i in df1.index:
    x =0
    for j in df1.loc[i,'words']:
        if j in negativewords:
            x += 1
            df1.loc[i,'NEGATIVE WORDS'] = x

In [ ]:
#Calculating Polarity Score
df1['POLARITY SCORE'] = (df1['POSITIVE WORDS'] - df1['NEGATIVE WORDS'])/ ((df1['POSITIVE WORDS'] + df1['NEGATIVE WORDS']) + 0.000001)


In [ ]:
df1['SUBJECTIVITY SCORE'] = (df1['POSITIVE WORDS'] - df1['NEGATIVE WORDS'])/ (df1['WORD COUNT'] + 0.000001)

In [ ]:
# calculate number of sentences in the article
df1['Number of Sentences'] = np.nan
for i in df1.index:
    x =0
    words = ''
    for j in df1.loc[i,'response']:
        words += j
        if words[-2:] == '. ':
            x += 1
            df1.at[i,'Number of Sentences'] = x

In [ ]:
#Average  Number of words in each sentence
df1['Average Number of Words Per Sentence'] = df1['WORD COUNT']/df1['Number of Sentences']

In [ ]:
df1['AVG SENTENCE LENGTH'] = df1['WORD COUNT']/df1['Number of Sentences']

In [ ]:
#Checking fog index
df1['FOG INDEX'] = 0.4*(df1['Average Number of Words Per Sentence'] + df1['Percentage OF COMPLEX WORDS'] )

In [ ]:
#Checking number of pronouns
df1['PERSONAL PRONOUNS'] = np.nan
for i in df1.index:
    x= 0
    for j in df1.loc[i,'words']:
        if j == 'US':
            x = x
        elif j.lower() in ["i", "you", "he", "she", "it", "we", "they", "them", "us", "him", "her", "his", "hers", "its", "theirs", "our", "your"]:
            x += 1
            df1.at[i,'PERSONAL PRONOUNS'] = x

In [ ]:
df1.to_csv('Output.csv')